In [ ]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.poly.base import backtest_df_only
from quantfreedom.poly.enums import *



np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

price_data = pd.read_hdf("../../tests/data/prices.hd5")
%load_ext autoreload
%autoreload 2

In [ ]:
rsi_ind = from_talib(
    func_name='rsi',
    nickname='rsi',
    price_data=price_data,
    input_names = ['open'],
    parameters = {'timeperiod': [15, 20]}
)
rsi_ind.data

In [ ]:
entries = rsi_ind.is_below(
    user_args=[40, 50],
    candle_ohlc="close",
)
entries

In [ ]:
account_state = AccountState()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01, 0.02]) / 100,
    sl_based_on_lookback=np.array([30, 40]) * 1.0,
    risk_reward=np.array([2, 5, 8]) * 1.0,
    leverage_type=np.array([LeverageType.Dynamic]),
    candle_body=np.array([CandleBody.Low]),
    entry_size_type=np.array([EntrySizeType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([100]) / 100,
)
backtest_settings = BacktestSettings(order_type=OrderType.Long)
exchange_settings = ExchangeSettings()

In [ ]:
strat_results_df, settings_results_df = backtest_df_only(
    account_state=account_state,
    order_settings_arrays=order_settings_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data,
    entries=entries,
)

In [ ]:
settings_results_df

In [ ]:
strat_results_df.head(50)